In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code/')

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import utils
from data_loading import ImageSequence

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


- https://www.pyimagesearch.com/2020/10/19/adversarial-images-and-attacks-with-keras-and-tensorflow/
- http://image-net.org/challenges/LSVRC/2014/browse-synsets

In [5]:
resnet = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


In [6]:
dataset_path = '/data/datasets/imagenet2012/'

In [7]:
train_generator = ImageSequence(dataset_path, 20, target_split='train', square_ims='center_pad')

  1%|          | 6/1001 [00:00<00:29, 33.86it/s]

generating splits


100%|██████████| 1001/1001 [00:09<00:00, 100.40it/s]


In [10]:
n_classes = train_generator.n_classes()

In [12]:
# change last layer to new config  
last_layer_config = resnet.layers[-1].get_config()
last_layer_config['units'] = n_classes
last_layer_config['name'] = 'predictions1001'
new_last_layer = keras.layers.Dense(**last_layer_config)#(model.layers[-2].output)

In [13]:
old_resnet = keras.Model(resnet.input, resnet.layers[-2].output)
new_model = keras.Model(old_resnet.input, new_last_layer(old_resnet.output))

In [14]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [15]:
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam

In [16]:
opti = Adam(lr=0.0001, 
            clipnorm=1)
loss = SparseCategoricalCrossentropy

new_model.compile(optimizer=opti, loss=SparseCategoricalCrossentropy())

In [17]:
train_generator = ImageSequence(dataset_path, 60, target_split='train', square_ims='center_pad')
val_generator = ImageSequence(dataset_path, 60, target_split='val', square_ims='center_pad')

  1%|          | 12/1001 [00:00<00:08, 116.87it/s]

generating splits


  1%|          | 12/1001 [00:00<00:08, 117.95it/s]

generating splits


100%|██████████| 1001/1001 [00:09<00:00, 105.09it/s]


##### First we are going to retrain only last layers

In [18]:
train_generator.on_epoch_end()
val_generator.on_epoch_end()

In [19]:
for l in new_model.layers[:-1]:
    l.trainable = False

In [ ]:
new_model.fit(
    train_generator,
    epochs=5,
    verbose=1,
    validation_data=val_generator,
    workers=6,
    use_multiprocessing=True)

Epoch 1/5
2484/9451 [======>.......................] - ETA: 42:57 - loss: 3.6262

/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


2497/9451 [======>.......................] - ETA: 42:52 - loss: 3.6205

/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


3516/9451 [==========>...................] - ETA: 37:04 - loss: 3.2667

/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


4303/9451 [============>.................] - ETA: 32:29 - loss: 3.0763

/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


4442/9451 [=============>................] - ETA: 31:40 - loss: 3.0476

In [ ]:
break

In [ ]:
from PIL import Image

In [ ]:
Image.open('/data/datasets/imagenet2012//n03933933/383.jpeg')

In [ ]:
np.random.shuffle([1,2,3])

In [ ]:
# im = plt.imread('../brad_samples/2097157039_872012429d.jpg')
# new_im = make_square(im)
# plt.figure()
# plt.imshow(new_im)

In [ ]:
import os

In [ ]:
# imgs_path = '/data/datasets/imagenet2012/n02319095/' #sea_urchin

valid_imgs = [x for x in os.listdir(imgs_path) if x.split('.')[-1] != 'json']

In [ ]:
# imgs_path = '/data/datasets/imagenet2012/n02319095/'
# 
# ims = []
# for i in valid_imgs:
#     i_path = '{}/{}'.format(imgs_path,i)
#     im_ar =  image.img_to_array(image.load_img(i_path, target_size=(224, 224)))
#     ims.append(im_ar)
# # img = 
# # x = image.img_to_array(img)
# # x = np.expand_dims(x, axis=0)
# x = np.array(ims)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=10)[0])

In [ ]:
# orig_pred = decode_predictions(preds, top=5)
# for pred in orig_pred:
#     print(pred)

In [ ]:
preds_codes = decode_predictions(np.expand_dims(np.zeros(1000), axis=0), top=1000)[0]

In [ ]:
preds_codes

In [ ]:
from PIL import Image

In [ ]:
# vi = valid_imgs[0]
# i_path = '{}/{}'.format(imgs_path,vi)
# i_path = '/home/ivan/Dropbox/scratch/hog_test.png'
i_path = '/home/ivan/scratch/marmot.jpg'
# i_path = '/home/ivan/scratch/guitarra_electrica.jpg'
# ii = image.load_img(i_path, target_size=(224, 224), )
im_pil_raw_squared = center_crop_square(i_path, desired_size=224)
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_ar_prec = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))[0]

In [ ]:
plt.figure()
plt.imshow(im_ar/255)
plt.figure()
plt.imshow(im_ar_prec)
plt.figure()
plt.imshow(depreprocess(im_ar_prec)/255)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

https://www.tensorflow.org/tutorials/generative/adversarial_fgsm

In [ ]:
EPS = 3 / 255.0
LR = 0.1
# EPS = 0.001
# LR = 0.05
optimizer = Adam(learning_rate=LR)
# lossfn = SparseCategoricalCrossentropy()
lossfn = CategoricalCrossentropy()

def clip_eps(tensor, eps):
    # clip the values of the tensor to a given range and return it
    return tf.clip_by_value(tensor, clip_value_min=-eps,
        clip_value_max=eps)

choices = 3

def generate_adversaries(model, baseImage, delta, classIdx, steps=50, verbose=False):
    ty = np.zeros(1000)
    ty[classIdx] = 1
#    # multi labels
#     aa = np.random.choice(np.arange(1000),choices)
#     for a in aa:
#         ty[a] = 1/choices
    ty = np.expand_dims(ty, axis=0)
    for step in range(0, steps):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            adversary = baseImage + preprocess_input(delta)
            predictions = model(adversary, training=False)
            loss = lossfn(tf.convert_to_tensor(ty), predictions)                        
            if verbose and step % 10 == 0:
                print("step: {}, loss: {}...".format(step,
                    loss.numpy()))
        # calculate the gradients of loss with respect to the perturbation vector
        gradients = tape.gradient(loss, delta)
        # update the weights, clip the perturbation vector, and
        # update its value
        optimizer.apply_gradients([(gradients, delta)])
        delta.assign_add(clip_eps(delta, eps=EPS))
    return delta            

In [ ]:
# ii = image.load_img(i_path, target_size=(224, 224))
# ii = image.load_img(i_path, target_size=(224, 224))
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_proc = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))

baseImage = tf.constant(im_proc, dtype=tf.float32)
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
deltaUpdated = generate_adversaries(model, baseImage, delta, 245, steps=200, verbose=True)

adverImage = (baseImage + deltaUpdated).numpy().squeeze()
adverImage = np.clip(adverImage, 0, 255).astype("uint8")

In [ ]:
ims_test = [baseImage, deltaUpdated]
for i in range(10):
    ims_test.append(baseImage + i* deltaUpdated)

ims_test = np.concatenate(ims_test,axis=0)
preds_all = model.predict(ims_test)

for ii, p in zip(ims_test, decode_predictions(preds_all, top=3)):
    plt.figure()
    plt.imshow(np.round(depreprocess(ii)).astype('int32'))
    print(p)
    plt.show()
    print('______________')

### Brute force loop

In [ ]:
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_proc = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))

noise_strengs = np.arange(10) / 2
# iterate over different noise categories
outputs = []
for noise_cat in tqdm(np.random.choice(np.arange(1000), 4, replace=True)):

    baseImage = tf.constant(im_proc, dtype=tf.float32)
    delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
#     print("[INFO] generating perturbation...")
    deltaUpdated = generate_adversaries(model, baseImage, delta, noise_cat, steps=300, verbose=False)

    adverImage = (baseImage + deltaUpdated).numpy().squeeze()
    adverImage = np.clip(adverImage, 0, 255).astype("uint8")

#     ims_test = [baseImage, deltaUpdated]
#     for ns in noise_strengs:
#         ims_test.append(baseImage + ns * deltaUpdated)
    ims_test = np.concatenate([baseImage + ns * deltaUpdated for ns in noise_strengs], axis=0)
    preds_all = model.predict(ims_test)
    preds_decoded = decode_predictions(preds_all, top=3)
    for ii, p in zip(ims_test, preds_decoded):
        plt.figure()
        plt.imshow(np.round(depreprocess(ii)).astype('int32'))
        print(p)
        plt.show()
        print('______________')
    
    for ns, pred in zip(noise_strengs, preds_decoded):
        sample = [i_path, noise_cat, ns, pred]
        outputs.append(sample)
        
            

In [ ]:
outputs